In [1]:
# Importamos librerías de análisis de datos
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

In [2]:
# Importamos utilidades y modelos de sklearn
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('/home/alvaro/Escritorio/TP2/train.csv', parse_dates=['fecha'])
df_test = pd.read_csv('/home/alvaro/Escritorio/TP2/test.csv')

In [4]:
def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0):
        return metroscubiertos
    else:
        return metrostotales

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns

dolar= pd.read_csv('/home/alvaro/Escritorio/TP2/Datos históricos USD_MXN.csv')
dolar.columns = ["fecha", "last", "opening", "max", "min", "std"]
dolar.drop(["std", "opening", "opening", "max", "min"], axis=1 ,inplace=True)
#Paso a formato fecha correctamente y luego creo columnas mes y año
dolar['date'] = pd.to_datetime(dolar['fecha'], format="%b %Y")
dolar.drop(["fecha"], axis=1, inplace=True)
dolar.columns= ['price', "date"]
#El csv ponia los numeros con punto en vez de coma.
dolar['price'] = dolar['price'].replace(',', '.', regex=True).astype(float)
dolar["year"] = dolar.date.dt.year
dolar['month'] = dolar.date.dt.month
dolar.drop(["date"], axis=1, inplace=True)

In [6]:
def setDataFrameWithDollarPrice(dfReceived, dolar):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month

    #Agrego la columna precio en dolar por propiedad, en el dataframe recibido.
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    dfReceived["dollar_price"] = dfReceived.apply(lambda row: row["precio"]/row["price"], axis=1)
    #Filtro dejando las cosas útiles.
    #Chequeado de no dejar alguna importante afuera
    dfReceived = dfReceived.filter(["id", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha',
                   'year', 'month',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad',
                   'centroscomercialescercanos', 'price','dollar_price'])
    print(dfReceived.columns)


    
    #Limpiando las propiedades tipo casa, no pueden no tener al menos una habitacion o baño y no pueden
    #tener metros no cubiertos
    filtro1 = dfReceived["tipodepropiedad"].isin(["Apartamento", "Casa", "Casa en condominio",
                                        "Casa uso de suelo", "Rancho", "Quinta Vacacional"])
    dfCasas = dfReceived[filtro1]
    dfCasas = dfCasas[np.isfinite(dfCasas['habitaciones'])]
    dfCasas = dfCasas[np.isfinite(dfCasas['banos'])]
    dfCasas = dfCasas[np.isfinite(dfCasas['metroscubiertos'])]
    dfCasas.garages.fillna(0, inplace=True)

    dfReceived = pd.merge(dfCasas, dfReceived, on=["id", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'year', 'month',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad',
                  'centroscomercialescercanos', 'price', 'dollar_price'], how='inner')
    #dfReceived = pd.merge(dfCasas, df, on=["id"], how = 'inner')
    dfReceived.rename(columns={"dollar_price": "precio"}, inplace = True)
    dfReceived.rename(columns={"price": "precioDelDolar"}, inplace = True)
    return dfReceived

In [7]:
df_train = setDataFrameWithDollarPrice(df, dolar)
df_train

Index(['id', 'tipodepropiedad', 'habitaciones', 'garages', 'banos', 'ciudad',
       'provincia', 'metroscubiertos', 'metrostotales', 'idzona', 'fecha',
       'year', 'month', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'antiguedad', 'centroscomercialescercanos', 'price',
       'dollar_price'],
      dtype='object')


,id,tipodepropiedad,habitaciones,garages,banos,ciudad,provincia,metroscubiertos,metrostotales,idzona,fecha,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio
0,254099,Apartamento,2.00,1.00,2.00,Benito Juárez,Distrito Federal,80.00,80.00,23533.00,2015-08-23,2015,8,0.00,0.00,0.00,0.00,nan,0.00,16.75,135683.67
1,53461,Casa en condominio,3.00,2.00,2.00,La Magdalena Contreras,Distrito Federal,268.00,180.00,24514.00,2013-06-28,2013,6,0.00,0.00,0.00,1.00,10.00,1.00,12.95,278078.17
2,247984,Casa,3.00,2.00,2.00,Tonalá,Jalisco,144.00,166.00,48551.00,2015-10-17,2015,10,0.00,0.00,0.00,0.00,5.00,0.00,16.50,72727.27
3,209067,Casa,2.00,1.00,1.00,Zinacantepec,Edo. de México,63.00,67.00,53666.00,2012-03-09,2012,3,0.00,0.00,0.00,1.00,1.00,1.00,12.81,50753.49
4,185997,Apartamento,2.00,1.00,1.00,Zapopan,Jalisco,95.00,95.00,47835.00,2016-06-07,2016,6,0.00,0.00,0.00,0.00,10.00,0.00,18.28,62910.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174992,119879,Casa,2.00,2.00,1.00,Zinacantepec,Edo. de México,67.00,nan,53666.00,2015-02-08,2015,2,0.00,0.00,0.00,0.00,0.00,0.00,14.95,43476.23
174993,259178,Casa,3.00,3.00,3.00,Toluca,Edo. de México,200.00,250.00,51954.00,2014-07-10,2014,7,0.00,0.00,0.00,1.00,0.00,1.00,13.22,146772.89
174994,131932,Apartamento,2.00,1.00,2.00,Benito Juárez,Distrito Federal,138.00,138.00,50003995.00,2015-03-03,2015,3,0.00,0.00,0.00,0.00,20.00,0.00,15.26,222778.44
174995,146867,Casa,4.00,0.00,4.00,Iztapalapa,Distrito Federal,235.00,137.00,24162.00,2014-12-26,2014,12,1.00,0.00,0.00,1.00,20.00,1.00,14.75,195945.49


In [8]:
def agregarAnioMesYPrecioDelDolarALaFecha(dfReceived):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    return dfReceived

In [9]:
# Droppeamos strings y columnas complejas y repetidas
drop_cols_train = ['fecha', 'idzona']
drop_cols_test = ['titulo', 'descripcion', 'direccion', 'lat', 'lng', 'fecha', 'idzona']
df_train = df_train.drop(drop_cols_train, axis=1).copy()

df_test = agregarAnioMesYPrecioDelDolarALaFecha(df_test)
df_test = df_test.drop(drop_cols_test, axis=1).copy()
print(f"Columnas ({len(df_train.columns)}): {df_train.columns.tolist()}")
df_train.head()

Columnas (19): ['id', 'tipodepropiedad', 'habitaciones', 'garages', 'banos', 'ciudad', 'provincia', 'metroscubiertos', 'metrostotales', 'year', 'month', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad', 'centroscomercialescercanos', 'precioDelDolar', 'precio']


,id,tipodepropiedad,habitaciones,garages,banos,ciudad,provincia,metroscubiertos,metrostotales,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio
0,254099,Apartamento,2.00,1.00,2.00,Benito Juárez,Distrito Federal,80.00,80.00,2015,8,0.00,0.00,0.00,0.00,nan,0.00,16.75,135683.67
1,53461,Casa en condominio,3.00,2.00,2.00,La Magdalena Contreras,Distrito Federal,268.00,180.00,2013,6,0.00,0.00,0.00,1.00,10.00,1.00,12.95,278078.17
2,247984,Casa,3.00,2.00,2.00,Tonalá,Jalisco,144.00,166.00,2015,10,0.00,0.00,0.00,0.00,5.00,0.00,16.50,72727.27
3,209067,Casa,2.00,1.00,1.00,Zinacantepec,Edo. de México,63.00,67.00,2012,3,0.00,0.00,0.00,1.00,1.00,1.00,12.81,50753.49
4,185997,Apartamento,2.00,1.00,1.00,Zapopan,Jalisco,95.00,95.00,2016,6,0.00,0.00,0.00,0.00,10.00,0.00,18.28,62910.28


In [10]:
df_test.dtypes

id                              int64
tipodepropiedad                object
ciudad                         object
provincia                      object
antiguedad                    float64
habitaciones                  float64
garages                       float64
banos                         float64
metroscubiertos               float64
metrostotales                 float64
gimnasio                      float64
usosmultiples                 float64
piscina                       float64
escuelascercanas              float64
centroscomercialescercanos    float64
year                            int64
month                           int64
price                         float64
dtype: object

In [11]:
display(df_train.isnull().sum())
numeric_columns_with_nulls = list(set(df_train.columns[df_train.isnull().sum() > 0].tolist()) - 
                                  set(['tipodepropiedad', 'ciudad', 'provincia']))
print(numeric_columns_with_nulls)

id                                0
tipodepropiedad                   0
habitaciones                      0
garages                           0
banos                             0
ciudad                          178
provincia                        48
metroscubiertos                   0
metrostotales                 36696
year                              0
month                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
antiguedad                    24006
centroscomercialescercanos        0
precioDelDolar                    0
precio                            0
dtype: int64

['antiguedad', 'metrostotales']


In [12]:
# Para los nulls numéricos, usar un Imputer con strategy mean (reemplazamos los NaN por el promedio)
# Para no leakear, spliteamos el dataset antes
df_train_ohe = pd.get_dummies(df_train, dummy_na=True)
print("First")
df_test_ohe = pd.get_dummies(df_test, drop_first=True)

First


In [13]:
X = df_train_ohe.drop("precio", axis=1)
y = df_train_ohe['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

display(df_train_ohe)

,id,habitaciones,garages,banos,metroscubiertos,metrostotales,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio,tipodepropiedad_Apartamento,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_nan,ciudad_Abasolo,ciudad_Acajete,ciudad_Acambay,ciudad_Acapulco de Juárez,ciudad_Acateno,ciudad_Acatzingo,ciudad_Acayucan,ciudad_Acolman,ciudad_Actopan,ciudad_Acuña,ciudad_Agua Dulce,ciudad_Agua Prieta,ciudad_Aguascalientes,ciudad_Ahome,ciudad_Ahuacatlán,ciudad_Ahualulco de Mercado,ciudad_Alamo Temapache,ciudad_Alamos,ciudad_Aldama,ciudad_Allende,ciudad_Almoloya de Alquisiras,ciudad_Almoloya de Juárez,ciudad_Altamira,ciudad_Altotonga,ciudad_Alvarado,ciudad_Alvaro Obregón,ciudad_Amacuzac,...,ciudad_Zapotlán de Juárez,ciudad_Zapotlán el Grande,ciudad_Zaragoza,ciudad_Zempoala,ciudad_Zihuatanejo de Azueta,ciudad_Zimatlán de Alvarez,ciudad_Zinacantepec,ciudad_ZirAndaro,ciudad_Zitácuaro,ciudad_Zumpahuacán,ciudad_Zumpango,ciudad_otra,ciudad_ácatlán,ciudad_ácaxochitlán,ciudad_ácámbaro,ciudad_áutlán de Navarro,ciudad_nan,provincia_Aguascalientes,provincia_Baja California Norte,provincia_Baja California Sur,provincia_Campeche,provincia_Chiapas,provincia_Chihuahua,provincia_Coahuila,provincia_Colima,provincia_Distrito Federal,provincia_Durango,provincia_Edo. de México,provincia_Guanajuato,provincia_Guerrero,provincia_Hidalgo,provincia_Jalisco,provincia_Michoacán,provincia_Morelos,provincia_Nayarit,provincia_Nuevo León,provincia_Oaxaca,provincia_Puebla,provincia_Querétaro,provincia_Quintana Roo,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_nan
0,254099,2.00,1.00,2.00,80.00,80.00,2015,8,0.00,0.00,0.00,0.00,nan,0.00,16.75,135683.67,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,53461,3.00,2.00,2.00,268.00,180.00,2013,6,0.00,0.00,0.00,1.00,10.00,1.00,12.95,278078.17,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,247984,3.00,2.00,2.00,144.00,166.00,2015,10,0.00,0.00,0.00,0.00,5.00,0.00,16.50,72727.27,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,209067,2.00,1.00,1.00,63.00,67.00,2012,3,0.00,0.00,0.00,1.00,1.00,1.00,12.81,50753.49,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,185997,2.00,1.00,1.00,95.00,95.00,2016,6,0.00,0.00,0.00,0.00,10.00,0.00,18.28,62910.28,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174992,119879,2.00,2.00,1.00,67.00,nan,2015,2,0.00,0.00,0.00,0.00,0.00,0.00,14.95,43476.23,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
174993,259178,3.00,3.00,3.00,200.00,250.00,2014,7,0.00,0.00,0.00,1.00,0.00,1.00,13.22,146772.89,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
# Con dummy_na=True, creamos la categoria "Es nulo" como una coordenada más de los one-hot vectors
# Comentar: ¿Hay leaks acá? ¿Sí / No? ¿Por qué?
#df2 = pd.get_dummies(df2, dummy_na=True)
print(f"Cantidad de columnas después del one-hot-encoding: {len(df_train_ohe.columns)}")
df_train_ohe.head(10)

Cantidad de columnas después del one-hot-encoding: 732


,id,habitaciones,garages,banos,metroscubiertos,metrostotales,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio,tipodepropiedad_Apartamento,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_nan,ciudad_Abasolo,ciudad_Acajete,ciudad_Acambay,ciudad_Acapulco de Juárez,ciudad_Acateno,ciudad_Acatzingo,ciudad_Acayucan,ciudad_Acolman,ciudad_Actopan,ciudad_Acuña,ciudad_Agua Dulce,ciudad_Agua Prieta,ciudad_Aguascalientes,ciudad_Ahome,ciudad_Ahuacatlán,ciudad_Ahualulco de Mercado,ciudad_Alamo Temapache,ciudad_Alamos,ciudad_Aldama,ciudad_Allende,ciudad_Almoloya de Alquisiras,ciudad_Almoloya de Juárez,ciudad_Altamira,ciudad_Altotonga,ciudad_Alvarado,ciudad_Alvaro Obregón,ciudad_Amacuzac,...,ciudad_Zapotlán de Juárez,ciudad_Zapotlán el Grande,ciudad_Zaragoza,ciudad_Zempoala,ciudad_Zihuatanejo de Azueta,ciudad_Zimatlán de Alvarez,ciudad_Zinacantepec,ciudad_ZirAndaro,ciudad_Zitácuaro,ciudad_Zumpahuacán,ciudad_Zumpango,ciudad_otra,ciudad_ácatlán,ciudad_ácaxochitlán,ciudad_ácámbaro,ciudad_áutlán de Navarro,ciudad_nan,provincia_Aguascalientes,provincia_Baja California Norte,provincia_Baja California Sur,provincia_Campeche,provincia_Chiapas,provincia_Chihuahua,provincia_Coahuila,provincia_Colima,provincia_Distrito Federal,provincia_Durango,provincia_Edo. de México,provincia_Guanajuato,provincia_Guerrero,provincia_Hidalgo,provincia_Jalisco,provincia_Michoacán,provincia_Morelos,provincia_Nayarit,provincia_Nuevo León,provincia_Oaxaca,provincia_Puebla,provincia_Querétaro,provincia_Quintana Roo,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_nan
0,254099,2.00,1.00,2.00,80.00,80.00,2015,8,0.00,0.00,0.00,0.00,nan,0.00,16.75,135683.67,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,53461,3.00,2.00,2.00,268.00,180.00,2013,6,0.00,0.00,0.00,1.00,10.00,1.00,12.95,278078.17,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,247984,3.00,2.00,2.00,144.00,166.00,2015,10,0.00,0.00,0.00,0.00,5.00,0.00,16.50,72727.27,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,209067,2.00,1.00,1.00,63.00,67.00,2012,3,0.00,0.00,0.00,1.00,1.00,1.00,12.81,50753.49,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,185997,2.00,1.00,1.00,95.00,95.00,2016,6,0.00,0.00,0.00,0.00,10.00,0.00,18.28,62910.28,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,126147,2.00,1.00,1.00,75.00,90.00,2014,3,0.00,0.00,0.00,0.00,5.00,1.00,13.06,84240.84,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,139233,3.00,1.00,2.00,140.00,160.00,2016,2,0.00,0.00,0.00,0.00,nan,0.00,18.14,63413.29,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,5013,4.00,2.00,3.00,293.00,293.00,2016,10,0.00,0.00,0.00,0.00,2.00,0.00,18.86,222646.31,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,44962,2.00,1.00,1.00,58.00,nan,2014,1,0.00

In [15]:
numeric_columns_with_nulls = list(set(df_train_ohe.columns[df_train_ohe.isnull().sum() > 0].tolist()) \
                                  - set(['tipodepropiedad', 'provincia', 'ciudad']))
print(numeric_columns_with_nulls)

['antiguedad', 'metrostotales']


In [16]:
from sklearn.impute import SimpleImputer
for c in numeric_columns_with_nulls:
    imp = SimpleImputer()
    X_train[c] = imp.fit_transform(X_train[[c]])
    X_test[c] = imp.transform(X_test[[c]])
    
print ("Finished")

Finished


In [17]:
from catboost import CatBoostRegressor



model = CatBoostRegressor(iterations=700,
                    learning_rate = 0.04,
                    depth=13,
                    eval_metric='RMSE',
                    bagging_temperature = 0.2,
                    metric_period = 75,
                    od_wait=100)
model.fit(X_train, y_train,
    eval_set=(X_test,y_test),
    #cat_features=categorical_features_pos,
    use_best_model=True,
    verbose=True)

0:	learn: 84120.5075416	test: 83411.7524943	best: 83411.7524943 (0)	total: 517ms	remaining: 6m 1s
75:	learn: 35026.5078808	test: 35367.3615615	best: 35367.3615615 (75)	total: 25.7s	remaining: 3m 31s
150:	learn: 32067.6773983	test: 32632.6039107	best: 32632.6039107 (150)	total: 51.1s	remaining: 3m 5s
225:	learn: 30834.4095927	test: 31693.4930289	best: 31693.4930289 (225)	total: 1m 14s	remaining: 2m 35s
300:	learn: 29940.9173898	test: 31087.2666388	best: 31087.2666388 (300)	total: 1m 40s	remaining: 2m 13s
375:	learn: 29339.5738898	test: 30746.7720601	best: 30746.7720601 (375)	total: 2m 6s	remaining: 1m 49s
450:	learn: 28754.3577599	test: 30462.0886831	best: 30462.0886831 (450)	total: 2m 33s	remaining: 1m 24s
525:	learn: 28176.2031238	test: 30234.4593896	best: 30234.4369821 (524)	total: 2m 59s	remaining: 59.4s
600:	learn: 27685.6740029	test: 30068.8011610	best: 30068.8011610 (600)	total: 3m 27s	remaining: 34.1s
675:	learn: 27227.2873119	test: 29932.1237734	best: 29932.1237734 (675)	total:

In [27]:
model.score(X_test, y_test)

0.808400846019489

In [28]:
df_test_ohe['tipodepropiedad_Garage'] = 0
df_test_ohe['tipodepropiedad_Hospedaje'] = 0
df_test_ohe['provincia_nan'] = 0
df_test_ohe['provincia_Aguascalientes'] = 0
df_test_ohe['tipodepropiedad_Apartamento'] = 0
df_test_ohe['tipodepropiedad_nan'] = 0

df_test_ohe.rename(columns={"price": "precioDelDolar"}, inplace = True)
lista_drop = ['ciudad_Río Blanco', 'ciudad_San Mateo Río Hondo', 'ciudad_Amanalco',
              'ciudad_Tetela del Volcán', 'ciudad_Sabinas Hidalgo', 'ciudad_Zaragoza',
              'ciudad_Atoyac', 'ciudad_Sabinas', 'ciudad_Lamadrid', 'ciudad_PuruAndiro', 
              'ciudad_Apatzingán', 'ciudad_Tlalpujahua', 'ciudad_Concordia',
              'ciudad_San Pedro Mixtepec -Dto. 26 -', 'ciudad_Iguala de la Independencia',
              'ciudad_Tepechitlán', 'ciudad_Tlalixtac de Cabrera', 'ciudad_Chalchicomula de Sesma', 
              'ciudad_Santa Cruz Acatepec', 'ciudad_Tempoal', 'ciudad_Macuspana', 'ciudad_Tecolotlán',
              'ciudad_Río Lagartos', 'ciudad_Santiago Miahuatlán', 'ciudad_Nautla', 
              'ciudad_Tepetitla de Lardizábal', 'ciudad_Batopilas', 'ciudad_Nazareno Etla', 
              'ciudad_San Gabriel', 'ciudad_Tecolutla', 'ciudad_Canatlán', 'ciudad_Villa Corzo', 
              'ciudad_nan', 'ciudad_Casas Grandes', 'ciudad_ZirAndaro', 'ciudad_Zacatepec', 
              'ciudad_San Gregorio Atzompa', 'ciudad_Escuinapa', 'ciudad_Cosío', 'ciudad_Calkiní', 
              'ciudad_Gral. Bravo', 'ciudad_Villa Guerrero', 'ciudad_Sayula', 'ciudad_Jáltipan', 
              'ciudad_Angostura', 'ciudad_La Trinitaria', 'ciudad_Tejupilco', 'ciudad_Empalme', 
              'ciudad_Puente de Ixtla', 'ciudad_Xalatlaco', 'ciudad_Tihuatlán', 'ciudad_Altotonga', 
              'ciudad_Guadalupe Victoria', 'ciudad_Alamos', 'ciudad_Perote', 'ciudad_Ixtaczoquitlán', 
              'ciudad_Purísima del Rincón', 'ciudad_Pánuco de Coronado', 'ciudad_Juárez Hidalgo', 
              'ciudad_Acatzingo', 'ciudad_Santa María del Oro', 'ciudad_Nanchital de Lázaro Cárdenas del Río',
              'ciudad_Tecozautla', 'ciudad_Dr. Belisario Domínguez', 'ciudad_Dzidzantún', 'ciudad_Mapastepec',
              'ciudad_Asientos', 'ciudad_Rafael Lara Grajales', 'ciudad_La Piedad', 'ciudad_Teocelo',
              'ciudad_Opichén', 'ciudad_Dzitás', 'ciudad_Umán', 'ciudad_San Francisco del Rincón', 
              'ciudad_Atemajac de Brizuela', 'ciudad_Nadadores', 'ciudad_Matehuala', 'ciudad_Jerécuaro',
              'ciudad_Chignahuapan', 'ciudad_áutlán de Navarro', 'ciudad_Ursulo Galván', 
              'ciudad_Ocotlán de Morelos', 'ciudad_Bella Vista', 'ciudad_Tezontepec de Aldama', 
              'ciudad_Jiquilpan', 'ciudad_Encarnación de Díaz', 'ciudad_Río Grande', 'ciudad_Aquismón',
              'ciudad_Dzoncauich', 'ciudad_Calvillo', 'ciudad_Huatusco', 'ciudad_Xaltocan', 'ciudad_Jamapa', 
              'ciudad_Pinal de Amoles', 'ciudad_Tototlán', 'ciudad_San Sebastián Tutla', 'ciudad_Homún', 
              'ciudad_Tlacotalpan', 'ciudad_Abasolo', 'ciudad_Tetecala', 'ciudad_Reyes Etla', 'ciudad_Acateno',
              'ciudad_Ecuandureo', 'ciudad_Magdalena', 'ciudad_Juchitán de Zaragoza', 'ciudad_Tenango del Aire', 
              'ciudad_Cuerámaro', 'ciudad_San Pablo Huitzo', 'ciudad_Moroleón', 'ciudad_San Raymundo Jalpan',
              'ciudad_Atempan', 'ciudad_El Fuerte', 'ciudad_Ixtlahuacán', 'ciudad_Ocosingo', 'ciudad_Rosario',
              'ciudad_Ahualulco de Mercado', 'ciudad_Paso de Ovejas', 'ciudad_Contepec', 'ciudad_Tlaxco',
              'ciudad_Ahuacatlán', 'ciudad_Ecatzingo', 'ciudad_Loreto', 'ciudad_Chiconcuac', 'ciudad_Colotlán', 
              'ciudad_Yuriria', 'ciudad_Marín', 'ciudad_Zimatlán de Alvarez', 'ciudad_ácaxochitlán', 
              'ciudad_Actopan', 'ciudad_Tlacolula de Matamoros', 'ciudad_Tamiahua', 'ciudad_Tianguismanalco',
              'ciudad_Alamo Temapache', 'ciudad_Santa Ana Nopalucan', 'ciudad_Totoltepec de Guerrero', 
              'ciudad_Ruíz', 'ciudad_ácámbaro', 'ciudad_Bocoyna', 'ciudad_Celestún', 'ciudad_Jiménez',
              'ciudad_Ciudad Fernández', 'ciudad_Tamazunchale', 'ciudad_Acayucan',
              'ciudad_Almoloya de Alquisiras', 'ciudad_Acuña', 'ciudad_ácatlán', 'ciudad_Soto la Marina',
              'ciudad_Tuxtla Chico', 'ciudad_Aramberri', 'ciudad_Tonanitla', 'ciudad_Tepeyanco', 
              'ciudad_San Felipe', 'ciudad_Jonacatepec', 'ciudad_Chilapa de Alvarez', 'ciudad_Gustavo Díaz Ordaz',
              'ciudad_Tizimín', 'ciudad_Chilchotla', 'ciudad_Cuaxomulco', 'ciudad_Jocotitlán', 'ciudad_Sinaloa',
              'ciudad_Timilpan', 'ciudad_Huandacareo', 'ciudad_San Juan Guelavía', 'ciudad_Catemaco', 
              'ciudad_Ixtlán del Río', 'ciudad_Urique', 'ciudad_Papantla', 'ciudad_Salvatierra', 
              'ciudad_Cihuatlán', 'ciudad_Atotonilco el Alto', 'ciudad_Otumba', 'ciudad_Colón', 
              'ciudad_San Martín de las Pirámides', 'ciudad_Omealca', 'ciudad_Tlaquiltenango', 
              'ciudad_Teteles de Avila Castillo', 'ciudad_Tepetitlán', 'ciudad_Lerdo de Tejada', 
              'ciudad_Tepalcatepec', 'ciudad_Elota', 'ciudad_Teuchitlán', 'ciudad_Cananea', 
              'ciudad_Chetumal', 'ciudad_Pedro Escobedo', 'ciudad_Taxco de Alarcón', 'ciudad_Villa de Arriaga',
              'ciudad_Tetepango', 'ciudad_Teapa', 'ciudad_Cocotitlán', 'ciudad_Chignautla', 'ciudad_Juanacatlán', 
              'ciudad_Gutiérrez Zamora', 'ciudad_Miahuatlán', 'ciudad_Camarón de Tejeda', 'ciudad_Tepeaca',
              'ciudad_Santiago Papasquiaro', 'ciudad_Mineral del Monte', 'ciudad_Ixhuatlancillo']
for ciudad in lista_drop:
    df_test_ohe[ciudad] = 0
df_test_ohe = df_test_ohe[list(X_train.columns.values)]

In [29]:
y_test

167055    56259.17
9813      80915.19
125958   191310.36
94896    250824.38
124802    68221.10
            ...   
132461   111603.54
28085    201869.87
124743   517948.72
18966    101093.36
67003     83927.14
Name: precio, Length: 17500, dtype: float64

In [30]:
y_false_pred = model.predict(X_test)
print(y_false_pred)

[ 73515.00715595  92913.08306589 183910.03536486 ... 519843.79179712
 106899.3531016   77440.16230401]


In [35]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_false_pred)

29902.363964044238

In [31]:
#Cosas para entregar
X_real_test = df_test_ohe
y_real_pred = model.predict(X_real_test)
y_real_pred

array([288208.75511778,  56179.65506632, 136479.67253702, ...,
        53424.32417074,  71727.47069806, 169744.85734471])

In [32]:
res = pd.DataFrame(y_real_pred, index= df_test_ohe.index, columns=['precio'])
res2 = res
res2['id'] = df_test_ohe["id"]
res2['precioDelDolar'] = df_test_ohe['precioDelDolar']
res = res.reset_index(drop=True)
res.set_index('id', inplace = True)

In [33]:
res['target'] = res['precio']*res['precioDelDolar']
res

,precio,precioDelDolar,target
id,,,
4941,288208.76,12.73,3669617.97
51775,56179.66,16.50,926964.31
115253,136479.67,15.38,2098716.16
299321,63455.77,15.35,973969.86
173570,41700.13,13.38,557960.23
...,...,...,...
75094,248795.70,16.92,4209449.08
171847,40119.57,18.86,756815.55
138313,53424.32,14.75,787955.36


In [ ]:
res.drop(['precio', 'precioDelDolar'], axis = 'columns', inplace = True)

In [57]:
res.reset_index()

,id,target
0,4941,3669617.97
1,51775,926964.31
2,115253,2098716.16
3,299321,973969.86
4,173570,557960.23
...,...,...
59995,75094,4209449.08
59996,171847,756815.55
59997,138313,787955.36
59998,271268,1353067.01


In [60]:
res.columns = ["target"]

,target
id,
4941,3669617.97
51775,926964.31
115253,2098716.16
299321,973969.86
173570,557960.23
...,...
75094,4209449.08
171847,756815.55
138313,787955.36


In [26]:
res.to_csv("/home/alvaro/Escritorio/TP2/CatBoost_grupo3.csv", header=True)